In [1]:
from __future__ import division, print_function, absolute_import
import tensorboard
import tflearn

n = 5

# Real-time data preprocessing
img_prep = tflearn.ImagePreprocessing()
img_prep.add_featurewise_zero_center(per_channel=True, mean=[0.53990436 , 0.4405486  , 0.39328504])

# Real-time data augmentation
img_aug = tflearn.ImageAugmentation()
img_aug.add_random_flip_leftright()
img_aug.add_random_crop([49, 49], padding=4)

# Building Residual Network
net = tflearn.input_data(shape=[None, 49, 49, 3],
                         data_preprocessing=img_prep,
                         data_augmentation=img_aug)
net = tflearn.conv_2d(net, 16, 3, regularizer='L2', weight_decay=0.0001)
net = tflearn.resnext_block(net, n, 16, 32)
net = tflearn.resnext_block(net, 1, 32, 32, downsample=True)
net = tflearn.resnext_block(net, n-1, 32, 32)
net = tflearn.resnext_block(net, 1, 64, 32, downsample=True)
net = tflearn.resnext_block(net, n-1, 64, 32)
net = tflearn.batch_normalization(net)
net = tflearn.activation(net, 'relu')
net = tflearn.global_avg_pool(net)
# Regression
net = tflearn.fully_connected(net, 11, activation='softmax')
opt = tflearn.Momentum(0.1, lr_decay=0.1, decay_step=32000, staircase=True)
net = tflearn.regression(net, optimizer=opt,
                         loss='categorical_crossentropy')
# Training
model = tflearn.DNN(net, checkpoint_path='Snapshots/model_resnext',
                    max_checkpoints=10, tensorboard_verbose=0, tensorboard_dir='Logs/',
                    clip_gradients=0.)

model.load('AffectNet_pretrained_model/model_resnext-96000')

curses is not supported on this machine (please install/reinstall curses for an optimal experience)
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.initializers.variance_scaling instead with distribution=uniform to get equivalent behavior.
Instructions for updating:
keep_dims is deprecated, use keepdims instead
Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Use standard file APIs to check for files with this prefix.
INFO:tensorflow:Restoring parameters from C:\Users\nishq\Desktop\Tensorflow-Bootcamp-master_backup\Infinite Analytics\Emotion Detection\AffectNet_pretrained_model\model_resnext-96000


In [2]:
import glob
import pandas as pd
import operator
import tensorflow as tf 
import numpy as np


output_dict = {}

output_dict['Filepath'] = []
output_dict['Label'] = []
output_dict['Neutral Confidence'] = []
output_dict['Happy Confidence'] = []
output_dict['Sad Confidence'] = []
output_dict['Surprise Confidence'] = []
output_dict['Fear Confidence'] = []
output_dict['Disgust Confidence'] = []
output_dict['Anger Confidence'] = []
output_dict['Contempt Confidence'] = []
output_dict['None Confidence'] = []
output_dict['Uncertain Confidence'] = []
output_dict['No-Face Confidence']  = []
sess= tf.Session()

html = """<!DOCTYPE html> 
<html>

<head>
    <link rel="stylesheet" href="style.css">
    <title>AffectNet Pretrained Model Testing</title>
</head>
<body>
    <center>
        <h1>AffectNet Pretrained Model testing</h1>
    </center>"""



# ----------------------------------------------------------------------------- Get all happy-labelled images
html+= """<center>
        <h1>Happy</h1>
    </center>
    
    <table align="center" >

        <tr>
            
            <th align="center" width=200 >Photo</th>
            <th align="center" width=200 >Emotion</th>
            <th align="center" width=200 >Emotion Confidence</th>
            
        </tr>"""


for filepath in glob.iglob('Datasets/AffectNet_subset/happy/*.jpg'):
    print(filepath)
    
    img = tf.read_file(filepath)
    img = tf.image.decode_jpeg(img, channels=3)
    img.set_shape([None, None, 3])
    img = tf.image.resize_images(img, (49, 49))
    img = img.eval(session=sess) # convert to numpy array
    img = np.expand_dims(img, 0) # make 'batch' of 1
    img = img/255.0
    
    prediction = model.predict(img)
    
    emotion_dict = {}
    emotion_dict['Neutral'] = prediction[0][0]
    emotion_dict['Happy'] = prediction[0][1]
    emotion_dict['Sad'] = prediction[0][2]
    emotion_dict['Surprise'] = prediction[0][3]
    emotion_dict['Fear'] = prediction[0][4]
    emotion_dict['Disgust'] = prediction[0][5]
    emotion_dict['Anger'] = prediction[0][6]
    emotion_dict['Contempt'] = prediction[0][7]
    emotion_dict['None'] = prediction[0][8]
    emotion_dict['Uncertain'] = prediction[0][9]
    emotion_dict['No-Face'] = prediction[0][10]
    
    output_dict['Filepath'].append(filepath)
    output_dict['Label'].append('Happy')
    output_dict['Neutral Confidence'].append(prediction[0][0])
    output_dict['Happy Confidence'].append(prediction[0][1])
    output_dict['Sad Confidence'].append(prediction[0][2])
    output_dict['Surprise Confidence'].append(prediction[0][3])
    output_dict['Fear Confidence'].append(prediction[0][4])
    output_dict['Disgust Confidence'].append(prediction[0][5])
    output_dict['Anger Confidence'].append(prediction[0][6])
    output_dict['Contempt Confidence'].append(prediction[0][7])
    output_dict['None Confidence'].append(prediction[0][8])
    output_dict['Uncertain Confidence'].append(prediction[0][9])
    output_dict['No-Face Confidence'].append(prediction[0][10]) 
    
    emotion = str(max(emotion_dict.items(), key=operator.itemgetter(1))[0])
    emotion_confidence = max(emotion_dict.items(), key=operator.itemgetter(1))[1] *100
    
    html+= """<tr align="center">

                <td><img src="""+'"'+str(filepath)+'"'+""" alt="" border=3 height=200 width=200></img></td>
                <td align="center" height=200 width=200 >"""+emotion+"""</td>
                <td align="center" height=200 width=200 >"""+str(round(emotion_confidence))+"""%</td>

            </tr>"""
    print(emotion)
    print(emotion_confidence)
    
html+="""</table>"""


# ----------------------------------------------------------------------------- Get all sad-labelled images

html+= """<center>
        <h1>Sad</h1>
    </center>
    
    <table align="center" >

        <tr>
            
            <th align="center" width=200 >Photo</th>
            <th align="center" width=200 >Emotion</th>
            <th align="center" width=200 >Emotion Confidence</th>
            
        </tr>"""


for filepath in glob.iglob('Datasets/AffectNet_subset/sad/*.jpg'):
    print(filepath)
    
    img = tf.read_file(filepath)
    img = tf.image.decode_jpeg(img, channels=3)
    img.set_shape([None, None, 3])
    img = tf.image.resize_images(img, (49, 49))
    img = img.eval(session=sess) # convert to numpy array
    img = np.expand_dims(img, 0) # make 'batch' of 1
    img = img/255.0
    
    prediction = model.predict(img)
    
    emotion_dict = {}
    emotion_dict['Neutral'] = prediction[0][0]
    emotion_dict['Happy'] = prediction[0][1]
    emotion_dict['Sad'] = prediction[0][2]
    emotion_dict['Surprise'] = prediction[0][3]
    emotion_dict['Fear'] = prediction[0][4]
    emotion_dict['Disgust'] = prediction[0][5]
    emotion_dict['Anger'] = prediction[0][6]
    emotion_dict['Contempt'] = prediction[0][7]
    emotion_dict['None'] = prediction[0][8]
    emotion_dict['Uncertain'] = prediction[0][9]
    emotion_dict['No-Face'] = prediction[0][10]
    
    output_dict['Filepath'].append(filepath)
    output_dict['Label'].append('Sad')
    output_dict['Neutral Confidence'].append(prediction[0][0])
    output_dict['Happy Confidence'].append(prediction[0][1])
    output_dict['Sad Confidence'].append(prediction[0][2])
    output_dict['Surprise Confidence'].append(prediction[0][3])
    output_dict['Fear Confidence'].append(prediction[0][4])
    output_dict['Disgust Confidence'].append(prediction[0][5])
    output_dict['Anger Confidence'].append(prediction[0][6])
    output_dict['Contempt Confidence'].append(prediction[0][7])
    output_dict['None Confidence'].append(prediction[0][8])
    output_dict['Uncertain Confidence'].append(prediction[0][9])
    output_dict['No-Face Confidence'].append(prediction[0][10]) 
    
    emotion = str(max(emotion_dict.items(), key=operator.itemgetter(1))[0])
    emotion_confidence = max(emotion_dict.items(), key=operator.itemgetter(1))[1] *100
    
    html+= """<tr align="center">

                <td><img src="""+'"'+str(filepath)+'"'+""" alt="" border=3 height=200 width=200></img></td>
                <td align="center" height=200 width=200 >"""+emotion+"""</td>
                <td align="center" height=200 width=200 >"""+str(round(emotion_confidence))+"""%</td>

            </tr>"""
    print(emotion)
    print(emotion_confidence)
    
html+="""</table>"""

# ----------------------------------------------------------------------------- Get all angry-labelled images

html+= """<center>
        <h1>Angry</h1>
    </center>
    
    <table align="center" >

        <tr>
            
            <th align="center" width=200 >Photo</th>
            <th align="center" width=200 >Emotion</th>
            <th align="center" width=200 >Emotion Confidence</th>
            
        </tr>"""


for filepath in glob.iglob('Datasets/AffectNet_subset/angry/*.jpg'):
    print(filepath)
    
    img = tf.read_file(filepath)
    img = tf.image.decode_jpeg(img, channels=3)
    img.set_shape([None, None, 3])
    img = tf.image.resize_images(img, (49, 49))
    img = img.eval(session=sess) # convert to numpy array
    img = np.expand_dims(img, 0) # make 'batch' of 1
    img = img/255.0
    
    prediction = model.predict(img)
    
    emotion_dict = {}
    emotion_dict['Neutral'] = prediction[0][0]
    emotion_dict['Happy'] = prediction[0][1]
    emotion_dict['Sad'] = prediction[0][2]
    emotion_dict['Surprise'] = prediction[0][3]
    emotion_dict['Fear'] = prediction[0][4]
    emotion_dict['Disgust'] = prediction[0][5]
    emotion_dict['Anger'] = prediction[0][6]
    emotion_dict['Contempt'] = prediction[0][7]
    emotion_dict['None'] = prediction[0][8]
    emotion_dict['Uncertain'] = prediction[0][9]
    emotion_dict['No-Face'] = prediction[0][10]
    
    output_dict['Filepath'].append(filepath)
    output_dict['Label'].append('Angry')
    output_dict['Neutral Confidence'].append(prediction[0][0])
    output_dict['Happy Confidence'].append(prediction[0][1])
    output_dict['Sad Confidence'].append(prediction[0][2])
    output_dict['Surprise Confidence'].append(prediction[0][3])
    output_dict['Fear Confidence'].append(prediction[0][4])
    output_dict['Disgust Confidence'].append(prediction[0][5])
    output_dict['Anger Confidence'].append(prediction[0][6])
    output_dict['Contempt Confidence'].append(prediction[0][7])
    output_dict['None Confidence'].append(prediction[0][8])
    output_dict['Uncertain Confidence'].append(prediction[0][9])
    output_dict['No-Face Confidence'].append(prediction[0][10]) 
    
    emotion = str(max(emotion_dict.items(), key=operator.itemgetter(1))[0])
    emotion_confidence = max(emotion_dict.items(), key=operator.itemgetter(1))[1] *100
    
    html+= """<tr align="center">

                <td><img src="""+'"'+str(filepath)+'"'+""" alt="" border=3 height=200 width=200></img></td>
                <td align="center" height=200 width=200 >"""+emotion+"""</td>
                <td align="center" height=200 width=200 >"""+str(round(emotion_confidence))+"""%</td>

            </tr>"""
    print(emotion)
    print(emotion_confidence)
    
html+="""</table>"""

# ----------------------------------------------------------------------------- Get all sad-labelled images

html+= """<center>
        <h1>Surprise</h1>
    </center>
    
    <table align="center" >

        <tr>
            
            <th align="center" width=200 >Photo</th>
            <th align="center" width=200 >Emotion</th>
            <th align="center" width=200 >Emotion Confidence</th>
            
        </tr>"""


for filepath in glob.iglob('Datasets/AffectNet_subset/surprise/*.jpg'):
    print(filepath)
    
    img = tf.read_file(filepath)
    img = tf.image.decode_jpeg(img, channels=3)
    img.set_shape([None, None, 3])
    img = tf.image.resize_images(img, (49, 49))
    img = img.eval(session=sess) # convert to numpy array
    img = np.expand_dims(img, 0) # make 'batch' of 1
    img = img/255.0
    
    prediction = model.predict(img)
    
    emotion_dict = {}
    emotion_dict['Neutral'] = prediction[0][0]
    emotion_dict['Happy'] = prediction[0][1]
    emotion_dict['Sad'] = prediction[0][2]
    emotion_dict['Surprise'] = prediction[0][3]
    emotion_dict['Fear'] = prediction[0][4]
    emotion_dict['Disgust'] = prediction[0][5]
    emotion_dict['Anger'] = prediction[0][6]
    emotion_dict['Contempt'] = prediction[0][7]
    emotion_dict['None'] = prediction[0][8]
    emotion_dict['Uncertain'] = prediction[0][9]
    emotion_dict['No-Face'] = prediction[0][10]
    
    output_dict['Filepath'].append(filepath)
    output_dict['Label'].append('Surprise')
    output_dict['Neutral Confidence'].append(prediction[0][0])
    output_dict['Happy Confidence'].append(prediction[0][1])
    output_dict['Sad Confidence'].append(prediction[0][2])
    output_dict['Surprise Confidence'].append(prediction[0][3])
    output_dict['Fear Confidence'].append(prediction[0][4])
    output_dict['Disgust Confidence'].append(prediction[0][5])
    output_dict['Anger Confidence'].append(prediction[0][6])
    output_dict['Contempt Confidence'].append(prediction[0][7])
    output_dict['None Confidence'].append(prediction[0][8])
    output_dict['Uncertain Confidence'].append(prediction[0][9])
    output_dict['No-Face Confidence'].append(prediction[0][10]) 
    
    emotion = str(max(emotion_dict.items(), key=operator.itemgetter(1))[0])
    emotion_confidence = max(emotion_dict.items(), key=operator.itemgetter(1))[1] *100
    
    html+= """<tr align="center">

                <td><img src="""+'"'+str(filepath)+'"'+""" alt="" border=3 height=200 width=200></img></td>
                <td align="center" height=200 width=200 >"""+emotion+"""</td>
                <td align="center" height=200 width=200 >"""+str(round(emotion_confidence))+"""%</td>

            </tr>"""
    print(emotion)
    print(emotion_confidence)
    
html+="""</table>"""

html+="""</body>"""
Html_file= open("AffectNet_observation_test_output.html","w",encoding="utf-8")
Html_file.write(html)
Html_file.close()

df = pd.DataFrame(output_dict)
df.to_csv('AffectNet_raw_test_data_output.csv')

Datasets/AffectNet_subset/happy\image0000626.jpg
Happy
95.28958201408386
Datasets/AffectNet_subset/happy\image0000629.jpg
No-Face
81.08367323875427
Datasets/AffectNet_subset/happy\image0000635.jpg
No-Face
50.49096941947937
Datasets/AffectNet_subset/happy\image0000636.jpg
Happy
49.02474284172058
Datasets/AffectNet_subset/happy\image0000637.jpg
Happy
68.50424408912659
Datasets/AffectNet_subset/happy\image0000638.jpg
Neutral
26.862531900405884
Datasets/AffectNet_subset/happy\image0000642.jpg
Happy
82.99608826637268
Datasets/AffectNet_subset/happy\image0000644.jpg
Happy
81.32333755493164
Datasets/AffectNet_subset/happy\image0000647.jpg


KeyboardInterrupt: 